## Invoke C++ from Python without bindings thanks to ROOT
* With ROOT, it's possible to use interactively C++ libraries from Python without the need to write bindings.
* The C++ entities known to the interpreter are exposed to Python transparently. 
* The following sources include a class, a function and a template.

## Header File

In [1]:
%%python
code='''
#include <iostream>
#include <typeinfo>

/// A trivial class
class A {
public:
   A();
   ~A();
};

/// A trivial function
int CountCharacters(const std::string s);

/// A trivial template
template<class T>
class B {
public:
   B()
   {
      std::cout << "The typeid name of the template argument is " << typeid(T).name() << std::endl;
   }
};
'''
with open('myLibrary.h','w') as f_out:
    f_out.write(code)

## Implementation

In [2]:
%%python
code='''
#include "myLibrary.h"    
A::A()
{
   std::cout << "This is the constructor of A" << std::endl;
}

A::~A()
{
   std::cout << "This is the destructor of A" << std::endl;
}

int CountCharacters(const std::string s)
{
   return s.size();
}
'''
with open('myLibrary.cc','w') as f_out:
    f_out.write(code)

## Creation of the Library
* It's trivial to create a shared object starting from the sources above:

In [3]:
%%bash
g++ -o libmyLibrary.so -shared -fPIC myLibrary.cc

In [4]:
%%bash
ls *so

libmyLibrary.so


* Now we'll see how easy it is to use this library from within Python thanks to ROOT.

## Interactivity without bindings
* In order to interact with the C++ entities contained in the library, we need to carry out to tasks:
  * We need to make known to the interpreter the interfaces. Concretely this means including one or more headers.
  * We need to make accessible to the interpreter the implementations of such C++ entities. Concretely this means loading the library.

In [5]:
import ROOT
ROOT.gInterpreter.ProcessLine('#include "myLibrary.h"')
ROOT.gSystem.Load("./libmyLibrary.so")

Welcome to JupyROOT 6.12/06


0

### We can now start exploring the content of the library. If you are wondering what a return code equal to 0 means, ROOT is telling us that the loading of the library happened without problems!

In [7]:
a = ROOT.A() ##the constructor of A
del a ##the destructor of A
b_doublePtr = ROOT.B("double*")() 
ROOT.CountCharacters("This interactivity without bindings is really impressive.")

57

This is the constructor of A
This is the destructor of A
The typeameof the template argument is Pd


## Interactivity within C++
* Interactive usage of C++ libraries is possible also in C++ mode.
* In this case, no wrapper is interposed between the statement written by the user and the call - ABI compatibility is leveraged. For example:

In [8]:
%%cpp
A a;

This is the constructor of A
